Company X has two plants, Plant1 and Plant2, which can produce Product1 and Product2

The cost of producing Product1 in Plant1 and Plant2 are 2 and 3 dollars, respectively

The cost of producing Product2 in Plant1 and Plant2 are 4 and 5 dollars, respectively

There are two customers, Customer1 and Customer2

Customer1 needs 200 of Product1 and 300 of Product2

Customer2 needs 500 of Product1 and 400 of Product2

Cost of transportation of Product1 from Plant1 to Customer1 and Customer2 is 0.2 and 0.3 dollars.

Cost of transportation of Product2 from Plant2 to Customer1 and Customer2 is 0.5 and 0.4 dollars.

What is the best production and transportation plan that minimizes the costs?

In [1]:
import pandas as pd
from dse_do_utils import DataManager, OptimizationEngine,ScenarioManager
from docplex.mp.model import Model
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

from docplex import __version__
__version__

'2.25.236'

# Define Data

In [2]:
Plants = pd.DataFrame(["Plant1","Plant2"], columns = ["PlantName"])

Customers = pd.DataFrame(["Customer1","Customer2"], columns = ["CustomerName"])

Products = pd.DataFrame(["Product1","Product2"], columns = ["ProductName"])

ProductionCosts = pd.DataFrame([["Plant1","Product1",2],
                                ["Plant2","Product1",3],
                                ["Plant1","Product2",4],
                                ["Plant2","Product2",5]], columns = ["PlantName","ProductName","ProductionCost"]) 


Demand = pd.DataFrame([["Customer1","Product1",200],
                       ["Customer1","Product2",300],
                       ["Customer2","Product1",500],
                       ["Customer2","Product2",400]], columns = ["CustomerName","ProductName","Demand"])

TransportationCosts = pd.DataFrame([["Plant1","Customer1","Product1",0.2],
                                    ["Plant1","Customer1","Product2",0.5],
                                    ["Plant1","Customer2","Product1",0.3],
                                    ["Plant1","Customer2","Product2",0.4],
                                    ["Plant2","Customer1","Product1",0.25],
                                    ["Plant2","Customer1","Product2",0.2],
                                    ["Plant2","Customer2","Product1",0.35],
                                    ["Plant2","Customer2","Product2",0.3]], columns = ["PlantName","CustomerName","ProductName","TransportationCost"]) 

ProductionCapacity = pd.DataFrame([["Plant1","Product1",500],
                                   ["Plant2","Product1",400],
                                   ["Plant1","Product2",300],
                                   ["Plant2","Product2",600]], columns = ["PlantName","ProductName","Capacity"]) 


inputs={}

inputs['Plants']=Plants
inputs['Customers']=Customers
inputs['Products']=Products
inputs['ProductionCosts']=ProductionCosts
inputs['Demand']=Demand
inputs['TransportationCosts']=TransportationCosts
inputs['ProductionCapacity']=ProductionCapacity

for i in inputs.values():
    display(i)

,PlantName
0,Plant1
1,Plant2


,CustomerName
0,Customer1
1,Customer2


,ProductName
0,Product1
1,Product2


,PlantName,ProductName,ProductionCost
0,Plant1,Product1,2
1,Plant2,Product1,3
2,Plant1,Product2,4
3,Plant2,Product2,5


,CustomerName,ProductName,Demand
0,Customer1,Product1,200
1,Customer1,Product2,300
2,Customer2,Product1,500
3,Customer2,Product2,400


,PlantName,CustomerName,ProductName,TransportationCost
0,Plant1,Customer1,Product1,0.20
1,Plant1,Customer1,Product2,0.50
2,Plant1,Customer2,Product1,0.30
3,Plant1,Customer2,Product2,0.40
4,Plant2,Customer1,Product1,0.25
5,Plant2,Customer1,Product2,0.20
6,Plant2,Customer2,Product1,0.35
7,Plant2,Customer2,Product2,0.30


,PlantName,ProductName,Capacity
0,Plant1,Product1,500
1,Plant2,Product1,400
2,Plant1,Product2,300
3,Plant2,Product2,600


# Optimization Model

In [3]:
def continuous_var_series(df, mdl,**kargs):
    return pd.Series(mdl.continuous_var_list(df.index, **kargs), index = df.index)

class CplexSum():
    """Function class that adds a series of dvars into a cplex sum expression.
    To be used as a custom aggregation in a groupby.
    Usage:
        df2 = df1.groupby(['a']).agg({'xDVar':CplexSum(engine.mdl)}).rename(columns={'xDVar':'expr'})

    Sums the dvars in the 'xDVar' column into an expression
    """
    def __init__(self, mdl):
        self.mdl = mdl
    def __call__(self, dvar_series):
        return self.mdl.sum(dvar_series)


def extract_solution(df, extract_dvar_names=None, drop_column_names=None, drop:bool=True):
    df = df.copy()
    """Generalized routine to extract a solution value. 
    Can remove the dvar column from the df to be able to have a clean df for export into scenario."""
    if extract_dvar_names is not None:
        for xDVarName in extract_dvar_names:
            if xDVarName in df.columns:
                df[f'{xDVarName}_Solution'] = [dvar.solution_value for dvar in df[xDVarName]]
                if drop:
                    df = df.drop([xDVarName], axis = 1)
    if drop and drop_column_names is not None:
        for column in drop_column_names:
            if column in df.columns:
                df = df.drop([column], axis = 1)
    return df  



Plants = inputs['Plants']
Customers = inputs['Customers']
Products = inputs['Products']
ProductionCosts= inputs['ProductionCosts']
Demand= inputs['Demand']
TransportationCosts= inputs['TransportationCosts']
ProductionCapacity= inputs['ProductionCapacity']

### Define Decision Variables

What are the production amount of Product1 and Product2 in either Plant1 or Plant2

How much of Product1 or Product2 should be sent from Plant1 and Plant2 to Customer1 and Customer2 

In [4]:
MODEL_NAME = 'SupplyChain'
SCENARIO_NAME = 'Base_Scenario' 
mdl = Model(MODEL_NAME)

In [5]:
Products.reset_index(drop=True)

,ProductName
0,Product1
1,Product2


In [6]:
ProductionPlan = pd.merge(Products.reset_index(drop=True),Plants.reset_index(drop=True),how="cross").set_index(["ProductName","PlantName"])
ProductionPlan["X_production"]= continuous_var_series(ProductionPlan, mdl, name="X_production", lb = 0)
display( ProductionPlan )

TransportationPlan = pd.merge(Products.reset_index(drop=True),Plants.reset_index(drop=True),how="cross").merge(Customers.reset_index(drop=True),how="cross").set_index(["ProductName","PlantName","CustomerName"])
TransportationPlan["X_transportation"]= continuous_var_series(TransportationPlan, mdl, name="X_transportation", lb = 0)
display( TransportationPlan )

X_production
ProductName PlantName                              
Product1    Plant1     X_production_Product1_Plant1
            Plant2     X_production_Product1_Plant2
Product2    Plant1     X_production_Product2_Plant1
            Plant2     X_production_Product2_Plant2

X_transportation
ProductName PlantName CustomerName                                            
Product1    Plant1    Customer1     X_transportation_Product1_Plant1_Customer1
                      Customer2     X_transportation_Product1_Plant1_Customer2
            Plant2    Customer1     X_transportation_Product1_Plant2_Customer1
                      Customer2     X_transportation_Product1_Plant2_Customer2
Product2    Plant1    Customer1     X_transportation_Product2_Plant1_Customer1
                      Customer2     X_transportation_Product2_Plant1_Customer2
            Plant2    Customer1     X_transportation_Product2_Plant2_Customer1
                      Customer2     X_transportation_Product2_Plant2_Customer2

### Define Constraints


We can't generate more than the plant capacity

In [7]:
Plant_Product_Capacity = pd.merge(ProductionPlan.reset_index(drop=False), ProductionCapacity, on = ["ProductName","PlantName"])
display(Plant_Product_Capacity)


for row in Plant_Product_Capacity.itertuples():
    mdl.add_constraint(row.X_production <= row.Capacity)

,ProductName,PlantName,X_production,Capacity
0,Product1,Plant1,X_production_Product1_Plant1,500
1,Product1,Plant2,X_production_Product1_Plant2,400
2,Product2,Plant1,X_production_Product2_Plant1,300
3,Product2,Plant2,X_production_Product2_Plant2,600


We must deliver at least the value of demand

In [8]:
Delivery = TransportationPlan.reset_index(drop = False)
Delivery = Delivery[["ProductName","CustomerName","X_transportation"]].groupby(["ProductName","CustomerName"]).agg(CplexSum(mdl)).reset_index(drop = False).rename(columns = {"X_transportation":"Total_Delivered"})
Delivery = pd.merge(Delivery, Demand.reset_index(drop = True), on = ["ProductName","CustomerName"] )

display(Delivery)

for row in Delivery.itertuples():
    mdl.add_constraint(row.Total_Delivered >= row.Demand)

,ProductName,CustomerName,Total_Delivered,Demand
0,Product1,Customer1,X_transportation_Product1_Plant1_Customer1+X_transportation_Product1_Plant2_Customer1,200
1,Product1,Customer2,X_transportation_Product1_Plant1_Customer2+X_transportation_Product1_Plant2_Customer2,500
2,Product2,Customer1,X_transportation_Product2_Plant1_Customer1+X_transportation_Product2_Plant2_Customer1,300
3,Product2,Customer2,X_transportation_Product2_Plant1_Customer2+X_transportation_Product2_Plant2_Customer2,400


We cannot ship more than what we produced in a plant

In [9]:
Shipment = TransportationPlan.reset_index(drop = False)
Shipment = Shipment[["ProductName","PlantName","X_transportation"]].groupby(["ProductName","PlantName"]).agg(CplexSum(mdl)).reset_index(drop = False).rename(columns = {"X_transportation":"Total_Shipped"})
Plant_Shipment = pd.merge(ProductionPlan.reset_index(drop= False), Shipment, on =[ "ProductName","PlantName"])


display(Plant_Shipment)

for row in Plant_Shipment.itertuples():
    mdl.add_constraint(row.Total_Shipped <= row.X_production)


,ProductName,PlantName,X_production,Total_Shipped
0,Product1,Plant1,X_production_Product1_Plant1,X_transportation_Product1_Plant1_Customer1+X_transportation_Product1_Plant1_Customer2
1,Product1,Plant2,X_production_Product1_Plant2,X_transportation_Product1_Plant2_Customer1+X_transportation_Product1_Plant2_Customer2
2,Product2,Plant1,X_production_Product2_Plant1,X_transportation_Product2_Plant1_Customer1+X_transportation_Product2_Plant1_Customer2
3,Product2,Plant2,X_production_Product2_Plant2,X_transportation_Product2_Plant2_Customer1+X_transportation_Product2_Plant2_Customer2


### KPIs and Objective Function

Production Cost

In [10]:
TotalProductionCosts = pd.merge(ProductionPlan.reset_index(drop = False), ProductionCosts.reset_index(drop = True), on = ["ProductName","PlantName"])
display(TotalProductionCosts)

Total_Production_Costs = mdl.sum(TotalProductionCosts.ProductionCost * TotalProductionCosts.X_production)
mdl.add_kpi(Total_Production_Costs   , "Total_Production_Costs")

,ProductName,PlantName,X_production,ProductionCost
0,Product1,Plant1,X_production_Product1_Plant1,2
1,Product1,Plant2,X_production_Product1_Plant2,3
2,Product2,Plant1,X_production_Product2_Plant1,4
3,Product2,Plant2,X_production_Product2_Plant2,5


DecisionKPI(name=Total_Production_Costs,expr=2X_production_Product1_Plant1+3X_production_Product1_Plant2+4X_p..)

Transportation Cost

In [11]:
TotalTransportationCosts = pd.merge(TransportationPlan.reset_index(drop = False), TransportationCosts.reset_index(drop = False), on = ["ProductName","CustomerName","PlantName"])
display(TotalTransportationCosts)

Total_Transportation_Costs = mdl.sum(TotalTransportationCosts.TransportationCost * TotalTransportationCosts.X_transportation)
mdl.add_kpi(Total_Transportation_Costs   , "Total_Transportation_Costs")

,ProductName,PlantName,CustomerName,X_transportation,index,TransportationCost
0,Product1,Plant1,Customer1,X_transportation_Product1_Plant1_Customer1,0,0.20
1,Product1,Plant1,Customer2,X_transportation_Product1_Plant1_Customer2,2,0.30
2,Product1,Plant2,Customer1,X_transportation_Product1_Plant2_Customer1,4,0.25
3,Product1,Plant2,Customer2,X_transportation_Product1_Plant2_Customer2,6,0.35
4,Product2,Plant1,Customer1,X_transportation_Product2_Plant1_Customer1,1,0.50
5,Product2,Plant1,Customer2,X_transportation_Product2_Plant1_Customer2,3,0.40
6,Product2,Plant2,Customer1,X_transportation_Product2_Plant2_Customer1,5,0.20
7,Product2,Plant2,Customer2,X_transportation_Product2_Plant2_Customer2,7,0.30


DecisionKPI(name=Total_Transportation_Costs,expr=0.200X_transportation_Product1_Plant1_Customer1+0.300X_transport..)

In [12]:
Total_Costs = Total_Transportation_Costs + Total_Production_Costs
mdl.add_kpi(Total_Costs   , "Total_Costs")

mdl.print_information()

Model: SupplyChain
 - number of variables: 12
   - binary=0, integer=0, continuous=12
 - number of constraints: 12
   - linear=12
 - parameters: defaults
 - objective: none
 - problem type is: LP


In [13]:
mdl.minimize(Total_Costs)
mdl.solve()

mdl.report()

* model SupplyChain solved with objective = 5210.000
*  KPI: Total_Production_Costs     = 4800.000
*  KPI: Total_Transportation_Costs = 410.000
*  KPI: Total_Costs                = 5210.000


In [14]:
### Postprocessing

In [15]:
ProductionPlan = extract_solution(Plant_Product_Capacity, extract_dvar_names= ['X_production'] ,drop=False)
display(ProductionPlan)

TransportationPlan = extract_solution(TransportationPlan, extract_dvar_names= ['X_transportation'] ,drop=False).reset_index(drop = False)
display(TransportationPlan)

Delivery = extract_solution(Delivery, extract_dvar_names= ['Total_Delivered'] ,drop=False)
display(Delivery)

,ProductName,PlantName,X_production,Capacity,X_production_Solution
0,Product1,Plant1,X_production_Product1_Plant1,500,500.0
1,Product1,Plant2,X_production_Product1_Plant2,400,200.0
2,Product2,Plant1,X_production_Product2_Plant1,300,300.0
3,Product2,Plant2,X_production_Product2_Plant2,600,400.0


,ProductName,PlantName,CustomerName,X_transportation,X_transportation_Solution
0,Product1,Plant1,Customer1,X_transportation_Product1_Plant1_Customer1,0.0
1,Product1,Plant1,Customer2,X_transportation_Product1_Plant1_Customer2,500.0
2,Product1,Plant2,Customer1,X_transportation_Product1_Plant2_Customer1,200.0
3,Product1,Plant2,Customer2,X_transportation_Product1_Plant2_Customer2,0.0
4,Product2,Plant1,Customer1,X_transportation_Product2_Plant1_Customer1,0.0
5,Product2,Plant1,Customer2,X_transportation_Product2_Plant1_Customer2,300.0
6,Product2,Plant2,Customer1,X_transportation_Product2_Plant2_Customer1,300.0
7,Product2,Plant2,Customer2,X_transportation_Product2_Plant2_Customer2,100.0


,ProductName,CustomerName,Total_Delivered,Demand,Total_Delivered_Solution
0,Product1,Customer1,X_transportation_Product1_Plant1_Customer1+X_transportation_Product1_Plant2_Customer1,200,200.0
1,Product1,Customer2,X_transportation_Product1_Plant1_Customer2+X_transportation_Product1_Plant2_Customer2,500,500.0
2,Product2,Customer1,X_transportation_Product2_Plant1_Customer1+X_transportation_Product2_Plant2_Customer1,300,300.0
3,Product2,Customer2,X_transportation_Product2_Plant1_Customer2+X_transportation_Product2_Plant2_Customer2,400,400.0


In [16]:
outputs={}
outputs['ProductionPlan']=ProductionPlan
outputs['TransportationPlan']=TransportationPlan
outputs['Delivery']=Delivery

In [17]:
io = {**inputs, **outputs}
writer = pd.ExcelWriter('test.xlsx', engine='openpyxl') 

for df_name, df in io.items():
    df.to_excel(writer, sheet_name=df_name)
writer.close()
;

''